# CONEXIÓN A MONGODB CON PYTHON
<img width = 70%; src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/MongoDB_Logo.svg/2560px-MongoDB_Logo.svg.png">

## En este notebook aprenderás a:
<div style="background-color: rgb(0, 143, 88);margin: 20px; padding: 20px; border-left: 9px solid #ffb200;">
<ul style="color: rgb(255, 255, 255);font-weight:bold;font-size:15px;">
    <li style="padding:10px;">Crear una conexión con una BBDD de MongoDB</li>
    <li style="padding:10px;">Ejecutar nuestras consultas y sentencias MQL con Python.</li>
    <li style="padding:10px;">Exportar consultas realizadas a una bbdd en MongoDB a JSON.</li>
</ul>
</div>

## INSTALACIÓN DE PSYCOPG2
<img width = 20%; src="https://jarroba.com/wp-content/uploads/2015/03/MongoPython.png">
Lo primero que haremos será instalar la librería de pymongo.

Para ello, puedes pegar esta instrucción en una celda de tipo código de tu notebook:
>```python
!pip install pymongo

*Enlace al repo de pypi: https://pypi.org/project/pymongo/*

In [1]:
!pip install pymongo

## IMPORTACIÓN

Una vez instalado el paquete, lo importamos para poder usarlo.

In [2]:
import pymongo
from pymongo import MongoClient

## CREANDO CONEXIÓN

Podemos crear la conexión vía string de conexión:

In [3]:
server ="localhost"
port = 27017
conexion_str =  f"mongodb://{server}:{str(port)}"
conexion = MongoClient(conexion_str)

O pasándole los parámetros de uno en uno:

In [4]:
conexion = MongoClient(
    host=server,
    port=port,
    username='',
    password=''
)

## Listar los **nombres** de las **bases de datos**

In [5]:
# listar bases de datos
db_names = conexion.list_database_names()
print(db_names)

['BBDD_PRUEBA', 'admin', 'config', 'local', 'prueba']


## Por cada base de datos listar el **número de colecciones** y diferenciar si son *vistas* o *colecciones*

In [6]:
# Por cada base de datos listar el número de colecciones y diferenciar si son vistas o colecciones
for db in db_names:
    if db not in ['local','admin','config']:
        print("*"*20)
        coll_names = conexion[db].list_collection_names()
        print(f"La base de datos {db} tiene {len(coll_names)} colecciones y son: {coll_names}")
        views_names = conexion[db].list_collections()
        if views_names:
            for element in list(views_names):
                print(f"La colección con nombre: {element['name']} es de tipo: {element['type']}")


********************
La base de datos BBDD_PRUEBA tiene 4 colecciones y son: ['system.views', 'users', 'peliculas', 'mujeres_adultas_por_ciudad']
La colección con nombre: system.views es de tipo: collection
La colección con nombre: users es de tipo: collection
La colección con nombre: peliculas es de tipo: collection
La colección con nombre: mujeres_adultas_por_ciudad es de tipo: view
********************
La base de datos prueba tiene 2 colecciones y son: ['miembro', 'users']
La colección con nombre: miembro es de tipo: collection
La colección con nombre: users es de tipo: collection


## Conectarnos a una base de datos concreta

In [7]:
db = conexion['BBDD_PRUEBA'] # usamos la bbdd de BBDD_PRUEBA

## Listar **los nombres de las colecciones** de una base de datos

In [8]:
# Listar los_nombres de las colecciones de una base de datos
collection_names = db.list_collection_names()
collection_names

['system.views', 'users', 'peliculas', 'mujeres_adultas_por_ciudad']

## Contar los documentos de una colección

In [9]:
# contar los documentos de la colección users
db[collection_names[1]].count_documents({})

2141

In [10]:
db['mujeres_adultas_por_ciudad'].count_documents({})

298

## Contar el número de documentos de cada colección

In [11]:
# contar el número de documentos de cada colección
for collection in collection_names:
    number_docs = db[collection].count_documents({})
    print(f'La colección  --{collection}--  tiene {number_docs} documentos.')

La colección  --system.views--  tiene 1 documentos.
La colección  --users--  tiene 2141 documentos.
La colección  --peliculas--  tiene 4 documentos.
La colección  --mujeres_adultas_por_ciudad--  tiene 298 documentos.


## **Encontrar** el *primer* documento

In [12]:
db.users.find_one()

{'_id': ObjectId('633b2087f2b6ef2a5eba5b44'),
 'name': 'BORJA',
 'age': 51,
 'gender': 'male',
 'phone': '982-666-667',
 'zip_code': 1000,
 'email': 'b.fernandez@miempresa.com',
 'city': 'San Sebastián',
 'nat': 'ES',
 'last_name': 'Fernández',
 'aficiones': ['Componer música', 'Pasear']}

## Contar los documentos de todxs los users que se llamen Manuel/manuel/MANUEL

In [13]:
# contar los documentos de todxs los users que se llamen Manuel
db.users.count_documents({"name":{'$regex':'manuel','$options':'i'}})

14

## Iterar sobre cada uno de los documentos:
- Si la edad (age) existe en el documento, Imprimir por pantalla: "X" tiene "y" años.
- Si el campo de aficiones existe en el documento, es de tipo dict y no hay valores para las aficiones de invierno, actualizar dichas aficiones con una afición escogida aleatoriamente entre ["ski","lectura","cocinar","cine"]
- Pasar todos los nombres a mayúsculas

In [14]:
import pprint
import random
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bson.objectid import ObjectId

aficiones_invierno = ["ski","lectura","cocinar","cine"]
for document in db.users.find():
    obj_id = document['_id']
    nombre = document['name'].upper()
    last_name = document['last_name']
    age = document['age']
    if 'age' in document:
        print(f"{nombre} {last_name} tiene {age} años.")
    if 'aficiones' in document:           
        aficiones = document['aficiones']
        if isinstance(aficiones,dict):
            if aficiones["invierno"] == None:
                aficion = random.choice(aficiones_invierno)
                #upsert: Update or Insert
                db.users.update_one({"_id": ObjectId(obj_id)},{"$set":{"aficiones.invierno":aficion}},upsert=True)
    #upsert: Update or Insert
    db.users.update_one({"_id": ObjectId(obj_id)},{"$set":{"name":nombre.upper()}},upsert=True)

BORJA Fernández tiene 51 años.
PEDRO Vicente tiene 28 años.
SILVIA Marin tiene 58 años.
SUSANA Dominguez tiene 66 años.
SEBASTIAN Mendoza tiene 55 años.
JOSEFA Navarro tiene 41 años.
MONICA Mendez tiene 59 años.
LUCAS Santana tiene 69 años.
JAVIER Vazquez tiene 33 años.
ESTHER Campos tiene 41 años.
LIDIA Gimenez tiene 71 años.
PAULA Santana tiene 51 años.
AMPARO Bravo tiene 44 años.
MINEA Makela tiene 29 años.
RAFAEL Santana tiene 42 años.
RUBEN Guerrero tiene 59 años.
RICARDO Gil tiene 64 años.
JESUS Lozano tiene 30 años.
ANDRES Leon tiene 60 años.
CARMELO Ramirez tiene 44 años.
NASH Voorneveld tiene 25 años.
BRETT Turner tiene 45 años.
ADRIANA Martin tiene 43 años.
MARIA Mora tiene 59 años.
AMPARO Molina tiene 57 años.
PHILIP Evans tiene 39 años.
ANGELES Rojas tiene 41 años.
MONICA Ibañez tiene 28 años.
MARCO Herrero tiene 33 años.
MIKAEL Heinonen tiene 56 años.
IGNACIO Nuñez tiene 42 años.
SHAUNI Boshoven tiene 49 años.
MONICA Moya tiene 59 años.
IIDA Tuomi tiene 69 años.
ALEJANDRO 

DANIEL Cabrera tiene 38 años.
ANNA Herrera tiene 33 años.
INMACULADA Lozano tiene 39 años.
JULIA Vazquez tiene 74 años.
PEDRO Vargas tiene 64 años.
NOELIA Lozano tiene 26 años.
CLARISSE Lemoine tiene 62 años.
CAROLINA Sanchez tiene 48 años.
IKER Alvarez tiene 76 años.
MIRIAM Crespo tiene 69 años.
JOSEFA Gallego tiene 36 años.
GUSTAVO Aguilar tiene 38 años.
BELEN Sanchez tiene 61 años.
BEGOÑA Reyes tiene 72 años.
MARIO Perez tiene 46 años.
DIEGO Gonzalez tiene 71 años.
HILLA Peltonen tiene 46 años.
JESUS Sanchez tiene 47 años.
TOMAS Alonso tiene 76 años.
JONATHAN Alonso tiene 56 años.
AGUSTIN Ibañez tiene 47 años.
ALEX Molina tiene 33 años.
CHRISTIAN Guerrero tiene 61 años.
ASUNCION Martinez tiene 61 años.
OLIVIA Justi tiene 58 años.
JORDI Santos tiene 51 años.
TRACY Williams tiene 37 años.
VICTORIA Gallego tiene 61 años.
SANDRA Rubio tiene 68 años.
LOUISA Gaillard tiene 48 años.
HECTOR Torres tiene 34 años.
FRANCISCO Cortes tiene 29 años.
AURORA Carmona tiene 73 años.
AGUSTIN Serrano t

MONTSERRAT Lozano tiene 28 años.
CLARA Sanz tiene 62 años.
MAEVE Van Altena tiene 60 años.
SOFIA Dominguez tiene 35 años.
LORENA Saez tiene 69 años.
JULIA Vargas tiene 31 años.
VICENTE Ramos tiene 27 años.
AAPO Ahonen tiene 28 años.
LOURDES Flores tiene 76 años.
SALVADOR Crespo tiene 44 años.
NEREA Prieto tiene 62 años.
RAQUEL Sanchez tiene 33 años.
MAGDALENA Soler tiene 65 años.
SOLEDAD Fuentes tiene 46 años.
DAVID Arias tiene 25 años.
SARA Savela tiene 36 años.
CONCEPCION Gallardo tiene 31 años.
CAROLINA Campos tiene 25 años.
AMPARO Santiago tiene 45 años.
JESSE Heino tiene 73 años.
ANTONIO Molina tiene 30 años.
EMILIA Morales tiene 74 años.
BIEKE Van Gaalen tiene 24 años.
ISAAC Cruz tiene 52 años.
MATHEW Gray tiene 66 años.
CHRISTIAN Ortiz tiene 44 años.
CELIA Suarez tiene 43 años.
SILVIA Molina tiene 72 años.
NEREA Hidalgo tiene 29 años.
RAMON Navarro tiene 67 años.
ELIAS Kivisto tiene 66 años.
DAVID Flores tiene 42 años.
LUCIA Pascual tiene 27 años.
WILLIAM Menard tiene 56 años.
P

IZZIE Lopez tiene 49 años.
ERNESTO Jimenez tiene 74 años.
LORENA Gallego tiene 70 años.
SEBASTIAN Gonzalez tiene 56 años.
INES Garcia tiene 53 años.
JAMIE Keuning tiene 32 años.
WAYNE Kelley tiene 59 años.
NELLA Bekking tiene 78 años.
OLIVER Juntunen tiene 32 años.
ALFONSO Garrido tiene 73 años.
ANTONIA Montero tiene 54 años.
CLARA Cruz tiene 56 años.
CHRISTIAN Carrasco tiene 33 años.
SAMU Suomi tiene 25 años.
CRISTOBAL Parra tiene 76 años.
SHERRI Little tiene 50 años.
ROSARIO Campos tiene 28 años.
NATALIA Ferrer tiene 72 años.
ROBERT Campbell tiene 70 años.
PAULINE Garcia tiene 70 años.
PATRICIA Martin tiene 38 años.
TERESA Santana tiene 60 años.
LIDIA Vazquez tiene 65 años.
MARCO Diaz tiene 45 años.
ERIN Rhodes tiene 62 años.
JOSE Peña tiene 55 años.
ANGELES Muñoz tiene 74 años.
ISMAEL Diez tiene 54 años.
TRINIDAD Diaz tiene 77 años.
SANTIAGO Marin tiene 43 años.
CRISTINA Gimenez tiene 63 años.
GUILLERMO Gomez tiene 47 años.
MONICA Nuñez tiene 63 años.
SANTIAGO Bravo tiene 44 años.
A

ARTTU Karjala tiene 69 años.
TRINIDAD Dominguez tiene 34 años.
RAFAEL Moya tiene 24 años.
ELLEN Jutila tiene 53 años.
AGUSTIN Mendez tiene 32 años.
GUILLERMO Carmona tiene 68 años.
VICTORIA Serrano tiene 59 años.
BENITO Soto tiene 57 años.
CONSUELO Carmona tiene 48 años.
FERNANDO Bravo tiene 63 años.
SHERRI Robertson tiene 42 años.
CARLOS Mendez tiene 51 años.
ADRIAN Reyes tiene 43 años.
DEBRA Black tiene 59 años.
ZOE Jackson tiene 43 años.
MOHAMED Benitez tiene 42 años.
JOAQUIN Serrano tiene 67 años.
CRISTINA Soto tiene 34 años.
GABRIEL Parra tiene 51 años.
RUBEN Montero tiene 50 años.
GORDON Ray tiene 42 años.
DAN Perry tiene 45 años.
MARGARITA Jimenez tiene 27 años.
CAROLINA Dominguez tiene 74 años.
EEMELI Korpi tiene 25 años.
GILBERT Jennings tiene 64 años.
JESUS Sanz tiene 52 años.
EMMI Peltola tiene 31 años.
EMILIO Jimenez tiene 76 años.
CHLOE Murray tiene 74 años.
GABRIEL Cano tiene 56 años.
DANIELA Aguilar tiene 44 años.
ENNI Kivisto tiene 62 años.
DANIEL Fernandez tiene 36 año

##  Utilizando las funciones de agregación obtener una lista de documentos de esta forma *[{nombre:x, num_apariciones:y}]*

In [15]:
# Utilizando las funciones de agregación obtener una 
# lista de documentos de esta forma [{nombre:x, num_apariciones:y}]
cuenta_nombres = list(db.users.aggregate([
    {"$group":{"_id":"$name", "num_apariciones":{"$sum":1}}},
    {"$project":{"_id":0,"nombre":"$_id","num_apariciones":1,}}
]))
cuenta_nombres

[{'num_apariciones': 3, 'nombre': 'NELLI'},
 {'num_apariciones': 8, 'nombre': 'PATRICIA'},
 {'num_apariciones': 1, 'nombre': 'PETRONELLA'},
 {'num_apariciones': 1, 'nombre': 'CELINA'},
 {'num_apariciones': 8, 'nombre': 'ANGELA'},
 {'num_apariciones': 2, 'nombre': 'JAMIE'},
 {'num_apariciones': 1, 'nombre': 'ALAN'},
 {'num_apariciones': 1, 'nombre': 'LETITIA'},
 {'num_apariciones': 8, 'nombre': 'TRINIDAD'},
 {'num_apariciones': 1, 'nombre': 'SUZANNE'},
 {'num_apariciones': 2, 'nombre': 'AADA'},
 {'num_apariciones': 3, 'nombre': 'EETU'},
 {'num_apariciones': 2, 'nombre': 'MAX'},
 {'num_apariciones': 1, 'nombre': 'JANET'},
 {'num_apariciones': 2, 'nombre': 'IZZIE'},
 {'num_apariciones': 2, 'nombre': 'JOONA'},
 {'num_apariciones': 11, 'nombre': 'RAMON'},
 {'num_apariciones': 1, 'nombre': 'TAMMY'},
 {'num_apariciones': 2, 'nombre': 'KRIN'},
 {'num_apariciones': 15, 'nombre': 'ALEX'},
 {'num_apariciones': 1, 'nombre': 'CLARISSE'},
 {'num_apariciones': 1, 'nombre': 'NASH'},
 {'num_apariciones

## Exportar el resultado en un **fichero JSON** que tenga por nombre algo como: **20220301203032_cuenta_nombres.json**

In [16]:
import pandas as pd
pd.DataFrame(cuenta_nombres)

,num_apariciones,nombre
0,3,NELLI
1,8,PATRICIA
2,1,PETRONELLA
3,1,CELINA
4,8,ANGELA
...,...,...
562,1,RORY
563,1,LENA
564,1,JORY
565,1,MARION


In [17]:
from datetime import datetime

# exportar el resultado en un fichero JSON que tenga por nombre algo como: 20220301203032_cuenta_nombres.json
import json
fecha_= datetime.now().strftime("%Y%m%d%H%M%S")
with open(f'{fecha_}_cuenta_nombres.json', 'w',encoding='utf8') as f:
    json.dump(cuenta_nombres, f, ensure_ascii=False)

## Leer de nuevo dicho archivo y obtener el número de apariciones de la persona cuyo nombre es MANUELA

In [18]:
# Leer de nuevo dicho archivo y obtener el número de apariciones de la persona cuyo nombre es MANUELA
with open(f'{fecha_}_cuenta_nombres.json',encoding="utf8") as json_file:
    data = json.load(json_file)

In [19]:
[doc["num_apariciones"] for doc in data if doc["nombre"] == "MANUELA"]

[8]

## Leer de nuevo el archivo **users.json**
- Imprimir la el **número** de documentos
- Incluir únicamente los documentos que tengan **más de 3 claves**.
- Si existe la llave edad, comprobar si es de tipo datetime.datetime y si no lo es parsearla como tal.
- **Si el campo email es nulo**, crearlo a partir del nombre+apellido@gmail.com

In [20]:
# Leer de archivo users.json
with open(f'users_to_mongo.json',encoding='utf8') as json_file:
    new_docs = json.load(json_file)
    print(len(new_docs))

9


In [21]:
new_docs

[{'name': 'Manuel',
  'last_name': 'González',
  'email': 'mgonzalez@cbnc.com',
  'age': '1992-12-12',
  'profesion': 'Pianista',
  'salario_mensual_actual': 2500,
  'formacion_academica': {'universidad': 'ehu',
   'fp': 'ils',
   'bachillerato': 'ils'},
  'exp_profesional': {'orquesta_sinfónica_euskadi': ['Pianista', 2]},
  'aficiones': {'verano': 'playa', 'invierno': ['ski', 'snow']},
  'coche': True,
  'direccion': {'municipio': 'irun',
   'cp': 20000,
   'barrio': 'meaka',
   'provincia': 'gipuzkoa',
   'ccaa': 'País Vasco'},
  'sexo': 'hombre',
  'estado_civil': 'casado',
  'hijos': 2,
  'idiomas': {'espaniol': ['nativo', 'intermedio']},
  'peliculas': [1, 3, 4]},
 {'name': 'Oscar',
  'last_name': 'González',
  'email': 'ogonzalez@cbnc.com',
  'age': '1990-12-31',
  'profesion': 'Barrendero',
  'salario_mensual_actual': 1650,
  'exp_profesional': {'ayto_municipal': ['Barrendero', 5]},
  'aficiones': {'verano': 'viajar', 'invierno': ['lectura']},
  'coche': False},
 {'name': 'Susan

In [22]:
from dateutil.relativedelta import relativedelta

docs_a_importar = []
print(f"La longitud del archivo es de: {len(new_docs)} documentos.")
for doc in new_docs:
    claves = doc.keys()
    if len(claves) > 3:
        if "age" in claves:
            fecha = doc["age"]
            doc["age"] = age = relativedelta(datetime.now(), datetime.strptime(fecha, "%Y-%m-%d")).years
        if not doc["email"]:
            doc["email"] = doc["name"].lower() + doc["last_name"].lower() + "@gmail.com"
        docs_a_importar.append(doc)
print(f"La longitud final de documentos a importar es dee: {len(docs_a_importar)} documentos.")

La longitud del archivo es de: 9 documentos.
La longitud final de documentos a importar es dee: 6 documentos.


## Insertar los nuevos documentos

In [23]:
# Insertar los nuevos documentos
db.users.insert_many(new_docs)

## Pasar todos los **apellidos a mayúsculas**

In [24]:
# Pasar todos los **nombres propios a mayúsculas**
db.users.update_many({},[{"$set":{"last_name":{"$toUpper":"$last_name"}}}])

## Eliminar todos los users que tengan 3 keys o menos

In [25]:
# eliminar todos los users que tengan 3 keys o menos
una_key_docs = []
for doc in db.users.find():
    if len(doc.keys()) <=3:
        _id = doc['_id']
        db.users.delete_one({"_id":ObjectId(_id)})

## Eliminar la vista "mujeres_adultas_por_ciudad"

In [26]:
# Eliminar la vista "mujeres_adultas_por_ciudad"
db.mujeres_adultas_por_ciudad.drop()

## Cerrar sesión

In [27]:
conexion.close()

<div style="background-color:  #FFCB92;margin: 20px; padding: 20px">
<b>¡Hasta la próxima pequeñx gran Egger!</b> 🐣 Ya estás un pasito más cerca de lograr tus metas!
</div>